In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from datasets import load_dataset

dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

In [2]:
ASEAN = ['Brunei', 'Cambodia', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam']
df_DA_ASEAN = df[(df['job_country'].isin(ASEAN)) & (df['job_title_short'] == 'Data Analyst')].copy()

df_DA_ASEAN = df_DA_ASEAN.dropna(subset='salary_year_avg')

df_DA_ASEAN_exploded = df_DA_ASEAN.explode('job_skills')

df_DA_ASEAN_exploded[['salary_year_avg', 'job_skills']].head(10)

,salary_year_avg,job_skills
3457,100500.0,python
3457,100500.0,sql
3457,100500.0,numpy
3457,100500.0,pandas
3457,100500.0,zoom
3457,100500.0,slack
7931,51014.0,python
7931,51014.0,r
7931,51014.0,sql
7931,51014.0,tableau
